In [29]:
# from google.colab import drive
# drive.mount('/content/drive')

In [30]:
# importer les package
import json
import string
import random
import nltk
import pandas as pd
import numpy as np
from matplotlib_inline.backend_inline import set_matplotlib_formats
import matplotlib.pyplot as plt
import re
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#lemmatization
from nltk.stem import WordNetLemmatizer
import joblib

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [31]:
#extraire le fichier intens.js
with open('intents.json') as f:
  df=json.load(f)


In [32]:
#afficher
df

{'intentions': [{'tag': 'salutations',
   'questions': ['Bonjour',
    'Bonsoir',
    'Salut',
    'Comment vas-tu?',
    'Comment ça va?',
    'Comment allez-vous?'],
   'reponses': ["Salut! Comment puis-je vous aider aujourd'hui?",
    'Salut! En quoi puis-je vous aider?',
    'En quoi puis-je vous aider?']},
  {'tag': 'infos_ecole',
   'questions': ["Est-ce que l'école est reconnue par l'état?",
    "L'école est-elle accréditée par les autorités compétentes?",
    "Est-ce que l'école est reconnue par le Ministère de l’Enseignement Supérieur",
    "êtes vous reconnue par l'état"],
   'reponses': ["L'école est reconnue par le Ministère de l’Enseignement Supérieur, de la Recherche et de l’Innovation du Sénégal sous le numéro RepSEN/Ensup-priv/AP/376-2021.",
    "L'école a obtenu sa reconnaissance officielle par le Ministère de l’Enseignement Supérieur, de la Recherche et de l’Innovation du Sénégal avec le numéro RepSEN/Ensup-priv/AP/376-2021.",
    "Nous sommes fiers de vous annoncer q

# traitement de donnée

In [33]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [34]:
# initilaiser le lemmatition
lemma=WordNetLemmatizer()
classes=[]# liste des label unque
words=[]# liste des differents mots unique
documentX=[]#liste des different questions(les label des question )
documentY=[]# liste des label de chaque questions( les question)
# labeliser les patterns et rajouter les questions dans le documentX et les etiquettes dans documentY
for intent in df["intentions"]:
  for questions in intent["questions"]:
    tkns= nltk.word_tokenize(questions)#tokeniser les questions(question)
    words.extend(tkns)# etendre les words (un append)
    documentX.append(questions)#rajouter les questions
    documentY.append(intent['tag'])#rajouter les label dans les document
  if intent["tag"] not in classes:
    classes.append(intent['tag'])#rajouter les label unique dans les class

words=[lemma.lemmatize(word.lower())for word in words if word is not string.punctuation]#lemmatiser words tout en s'assurant  que les mots sont lowercase  et les ponctuation
words=sorted(set(words))#récuperer les mots unique et ordonner en ordre alphabetique
classes =sorted(set(classes))# récuperer les label unique et ordonner les en ordre alphabetique




In [35]:
documentY[0]
#classes.index()

'salutations'

In [36]:
output_vec=[0]*len(classes)

In [37]:
output_vec[classes.index(documentY[0])]=1
print(output_vec)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


## vectorisation

In [38]:
from sklearn.feature_extraction.text import CountVectorizer

train_data = []  # liste des données trains
output_vec = [0] * len(classes)  # vecteur en sortie

# vectoriser les phrases de nos questions
vectorizer = CountVectorizer(vocabulary=words)  # initialiser le vectoriser
sentences_object = vectorizer.fit_transform(documentX)  # vectoriser documentX
sentences_array = sentences_object.toarray()  # convertir en array

for idx, doc in enumerate(documentX):
    output_row = [0] * len(classes)
    output_row[classes.index(documentY[idx])] = 1 # convertir output_vec en liste et l'afficher en output_row
    train_data.append([sentences_array[idx], output_row])  # rajouter la liste contenant le vecteur phrase et son label sur training data

random.shuffle(train_data)  # faire un shuffling de train_data
train_data = np.array(train_data, dtype=object)  # convertir train_data en array

# fractionner les donnees train_data
x = np.array(train_data[:, 0].tolist())
y = np.array(train_data[:, 1].tolist())


In [39]:
x.shape

(125, 202)

In [40]:
# definir l'optimizer
import tensorflow as tf
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense , Embedding , LSTM,Bidirectional,Dropout,GRU

# splitter mes données

In [41]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
#definir les parametres
input_shape=(len(x[0]))
output_shape=len(y[0])
#architecture de notre modele
from tensorflow.keras.models import Sequential
model=Sequential()

# Diviser les données en ensembles d'entraînement (80%) et de test + validation (20%)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

# Definir le model
model = Sequential()
model.add(Dense(128, input_shape=(input_shape,), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation='softmax')) 

# Compiler le model
optimizer = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# afficher le  summary
model.summary()

# Implement early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# entrainement
history = model.fit(x_train, y_train, epochs=200, validation_data=(x_val, y_val), callbacks=[early_stopping])


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 128)               25984     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 28)                1820      
                                                                 
Total params: 36060 (140.86 KB)
Trainable params: 36060 (140.86 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/200
4/4 

In [42]:
model.evaluate(x,y)

4/4 [==============================] - 0s 7ms/step - loss: 0.2437 - accuracy: 0.9360


[0.24370700120925903, 0.9359999895095825]

In [43]:
# import pickle

# with open('modele_chatbot.pkl', 'wb') as model_file:
#     pickle.dump(model, model_file)

# #extraire le fichier intens.js
# with open('intents.json') as f:
#   df=json.load(f)
from keras.models import save_model

# Assuming your Keras model is named 'model'
model.save("model_saved.h5")



c:\Users\hp\anaconda3\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [44]:
# Fonction tokenisation et lemmatisant les questions entres par l'utilisateur
def tk_lm_func(text):
  tkns=nltk.word_tokenize(text)
  tkns=[lemma.lemmatize(word) for word in tkns]
  return tkns
# fonction vectorisant la question entrée par l'utilisateur
def vectorizer_func(text,vocab):
  tkns=tk_lm_func(text)
  sent_vec=[0]*len(vocab)
  for w in tkns:
    for  idx,word in enumerate(vocab):
      if word==w:
        sent_vec[idx]=1
  return np.array(sent_vec)
from scipy.special import softmax

def Pred_funct(texte, vocab, labels):
    sent_vec = vectorizer_func(texte, vocab)
    result = model.predict(np.array([sent_vec]))
    
    # Appliquer la fonction softmax pour obtenir des probabilités
    prob_scores = softmax(result, axis=1)
    
    # Récupérer la classe prédite et le score de confiance associé
    predicted_class = np.argmax(prob_scores)
    tag = labels[predicted_class]
    confidence = prob_scores[0, predicted_class]
    
    return tag, confidence

def get_res(tag,fJson):
  list_intents= fJson['intentions']
  for i in list_intents:
    if i["tag"] ==tag:
      ourResult=random.choice(i['reponses'])
      break
  return ourResult




In [45]:
# #lancer le chatting
# while True:
#   Newmessage=input('')
#   if Newmessage.lower()=='quitter':
#     break
#   print("User:",Newmessage)
#   tag=Pred_funct(Newmessage,words,classes)
#   result=get_res(tag,df)
#   print('chatbot:',result)

In [46]:
import tkinter
from tkinter import *
import tkinter as tk
from tkinter import Text, Scrollbar, Button, Entry

def send_message():
    msg = EntryBox.get("1.0", 'end-1c').strip()
    EntryBox.delete("0.0", tk.END)
    if msg != '':
        ChatBox.config(state=tk.NORMAL)
        ChatBox.insert(tk.END, "You: " + msg + '\n\n')
        ChatBox.config(foreground="#446665", font=("Verdana", 12))
        
        # Appeler la fonction de prédiction
        tag, confidence = Pred_funct(msg, words, classes)
        res = get_res(tag, df)
        
        # Afficher la réponse, le score de confiance et le tag
        response_text = "Bot: {}\nConfidence: {:.5f}%\nTag: {}\n\n".format(res, confidence * 100, tag)
        ChatBox.insert(tk.END, response_text)
        
        ChatBox.config(state=tk.DISABLED)
        ChatBox.yview(tk.END)

# Initialiser la fenêtre principale
root = tk.Tk()
root.title("Chatbot")
root.geometry("400x500")
root.resizable(width=False, height=False)

#Créer la fenêtre de discussion
ChatBox = Text(root, bd=0, bg="white", height="30", width="50", font="Arial")
ChatBox.config(state=tk.DISABLED)

#Attacher la barre de défilement à la fenêtre de discussion
scrollbar = Scrollbar(root, command=ChatBox.yview, cursor="heart")
ChatBox['yscrollcommand'] = scrollbar.set

# Créer le bouton pour envoyer un message
SendButton = Button(root, font=("Verdana", 12, 'bold'),
                    text="Send", width="10", height="4",
                    bd=0, bg="#f9a602", activebackground="#3c9d9b",
                    fg='#000000', command=send_message)

#Créer la boîte pour entrer le message".
EntryBox = Text(root, bd=0, bg="white", width="40", height="4", font="Arial")

#Placez tous les composants sur l'écran
scrollbar.place(x=376, y=6, height=386)
ChatBox.place(x=6, y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

# Lancer la boucle principale de l'interface graphique
root.mainloop()

